## Do calculations module this number

In [1]:
M = 2^256 - 2^32 - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 -1

## Note the modulus is prime

In [2]:
M.is_prime()

True

## GF Creates a finite field

Don't worry about what a "finite field" is yet. It's not complicated. For now, it means "do calculation module M".

In [3]:
F = GF(2^256 - 2^32 - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 -1) 

Elliptic curves have the form $$y^2 = x^3 + ax + b$$
Sage uses the function `EllipticCurve(F, [a,b])` to define an an elliptic curve over the field F.
We already defined our field above (I'll talk about this field later, ignore it for now).
The secp256k1 has the form $$y^2 = x^3 + 7$$ (`a` is zero in this case).
The following will define the secp256k1 curve in sage

In [4]:
secp256k1 = EllipticCurve(F, [0,7])  

Let's look at sage's description of `secp256k1`

In [5]:
secp256k1

Elliptic Curve defined by y^2 = x^3 + 7 over Finite Field of size 115792089237316195423570985008687907853269984665640564039457584007908834671663

## The point `G_secp` is the generator for our group
 Since our operations are "on the elliptic curve" we need to tell sage that `x,y` is point on the curve.
 Since `secp256k1` is our curve object, use `secp256k1([x,y])` to get the curve point `x,y`.
 We get the values of the generator from the bitcoin wikipedia: https://en.bitcoin.it/wiki/Secp256k1

In [6]:
Gx=0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy=0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
G_secp = secp256k1([Gx, Gy])

Let look at sage's description of `G_secp`

In [7]:
G_secp

(55066263022277343669578718895168534326250603453777594175500187360389116729240 : 32670510020758816978083085130507043184471273380659243275938904335757337482424 : 1)

Wait, what?!? G is a point on the two dimentional curve $$y^2=x^3+7$$. But there are three elements to the point `G`. What's going on? It's called "homogenious coordinates". Let's leave them for another discussion. For now, just ignore the last component, except for one special case we'll talk about shortly.

The other two components are just the base ten representation of Gx, and Gy.

## The order of the group generated by G is not the same as the modulus

Note: explain how the group is generated by G

In [8]:
O = G_secp.order()

In [9]:
O == M

False

In [10]:
O

115792089237316195423570985008687907852837564279074904382605163141518161494337

In [11]:
M

115792089237316195423570985008687907853269984665640564039457584007908834671663

## What happens if the secret key is bigger than the order?
## It wraps around!

In [12]:
(O+1)*G_secp == G_secp

True

In [13]:
(O+2)*G_secp == 2*G_secp

True

## Generate a public key from a secret key

In [14]:
sk = 99999999999999999999999999999999999999999999999 # Not so secret

In [15]:
pk = sk*G_secp

In [16]:
pk

(28831862384762605154181516693812681466527496118086978914206801579353648892902 : 98051136482972846566922474755707477614442518354856504335042501106263341115100 : 1)

## Confirm it's a point on the curve

In [17]:
pk[0]^3 - pk[1]^2 + 7

0

## Yes, it's zero, as expected. Note that sage knows that `pk` is a point on the curve, so it does the calculation module M.

## Let's convert that public key to the "uncompressed point" form. This is just the hex encoding of the x component and y component concatentated together. Then we add the prefix "4" so we can distinguish this encoding from other encodings - like the compressed encoding.

In [18]:
encoded_pk = '4' + hex(Integer(pk[0])) + hex(Integer(pk[1]))

In [19]:
encoded_pk

'43fbe417ceeefde4f069edbc5f5e5a3134ec956523ab4ae94ea0929c2e53f8be6d8c6fa013ecf0fc315b2cdbe7ec5cb8e3a7ece104b8181c893fae92ad6f3a6dc'

## Two notes: the `+` above is string concatenation, not addition. Als, the cast to `Integer` is needed because `pk[0]` is an element of the finite field F and `hex` doesn't know what to do with this.

TBD: Now look at Ed25519. What's the modulus order? Is the order bigger than the modulus? Does this make sense? Is the order bigger than 2^256? Also talk about how this is used for signing things, not encrypting things. Show how signing works.

Note the [0,48662,0,1] for and explain them. Also note this is weierstrass form and usually Ed25519 is expressed in another form.

In [20]:
ed25519 = EllipticCurve(GF(2^255-19),[0,486662,0,1,0]) 

## The generator (base point) for ed25519 (according to https://crypto.stackexchange.com/questions/27392/base-point-in-ed25519) is
[15112221349535400772501151409588531511454012693041857206046113283949847762202, 46316835694926478169428394003475163141307993866256225615783033603165251855960]

That didn't work. According to this (https://github.com/paulmillr/noble-ed25519) it's
 [9, 14781619447589544791020593568409986887264606134616475288964881837755586237401]

In [21]:
G_ed = ed25519([9, 14781619447589544791020593568409986887264606134616475288964881837755586237401])

In [22]:
O = G_ed.order()

In [23]:
O

7237005577332262213973186563042994240857116359379907606001950938285454250989

In [24]:
ed15519_prob_miss = (2^256 - G_ed.order())/(2^256)
secp256k1_prob_miss = (2^256 - G_secp.order())/(2^256)

In [25]:
numerical_approx(ed15519_prob_miss)

0.937500000000000

In [26]:
numerical_approx(secp256k1_prob_miss)

3.73445534504013e-39

In [27]:
numerical_approx(log(G_ed.order(),2))

252.000000000000

In [28]:
numerical_approx(log(G_secp.order(),2))

256.000000000000

## I don't understand why we check for "valid" secp256k1 secret keys but not ed25519 secret keys. The order of secp256k1 keys is much larger than ed25519 keys (ed25519 is only about 2^252) I understand since it "wraps around" distinct 256 bit secret keys may in fact be the same secret key, but the problem is greater than ed25519 than secp256k1

The only reference I see is here https://en.bitcoin.it/wiki/Private_key#Range_of_valid_ECDSA_private_keys

The reason the range isn't checked for ed25519 is the secret key isn't used directly. Instead, it's hashed and some bits are masked off. Here's the calculation from the ed2519-donna library
```c++
ed25519_extsk(hash_512bits extsk, const ed25519_secret_key sk) {
    ed25519_hash(extsk, sk, 32);
    extsk[0] &= 248;
    extsk[31] &= 127;
    extsk[31] |= 64;
}
```

In [18]:
"{0:b}".format(248) # force these zero bits off in the first byte

'11111000'

In [19]:
"{0:b}".format(127) #force bit seven off

'1111111'

In [20]:
"{0:b}".format(64) # force bit six on

'1000000'